In [3]:
from threading import Thread, Condition, Lock
from time import sleep
import random
import copy

In [4]:
class Environment(object):
    #multiplying by this wil be equivalen to (%): [0, 25, 45, 60, 70, 75, 78, 80, 80]
    _concurrent_loss= [1, 0.75, 0.55, 0.4, 0.3, 0.25, 0.22, 0.2, 0.2]

    _audience = [
        [2,   1.3,    1],
        [1.3, 0.9, 0.75],
        [1,  0.75, 0.47]
    ]

    #0 will due to a total score of 0 to this match so will be discarded
    _hour_coef = [
        [  0, 0.55, 0.45,   0],
        [  0,  0.7, 0.75,   0],
        [  0,  0.8,  0.5,   0],
        [0.4,    1,    1, 0.4]
    ]

    _categories = ['A', 'B', 'C']
    _days = ['V', 'S', 'D', 'L']
    _hours = [12, 16, 18, 20]

    _idx_days = range(len(_days))
    _idx_hours = range(len(_hours))


class Match(object):
    def __init__(self, idx_homeCat, idx_awayCat, idx_day, idx_hour):
        self._hCat = idx_homeCat
        self._aCat = idx_awayCat
        self._d = idx_day
        self._h = idx_hour

    def get_raw_time(self):
        return [self._d, self._h]

    def get_away_category(self):
        return Environment._categories[self._aCat]

    def get_home_category(self):
        return Environment._categories[self._hCat]

    def get_base(self):
        return Environment._audience[self._hCat][self._aCat]

    def get_hour_coef(self):
        return Environment._hour_coef[self._h][self._d]

    def get_score(self):
        return self.get_base() * self.get_hour_coef()


class GenotypeTimetable(object):
    def __init__(self, num_matches=20, chain=None):
        '''
        Genotype representation, list of lists: [[1, 2]] ([[D, H], [D, H]])
        where in pos 0 is the index of day, and in pos 1 the index of hour
        '''
        if chain != None:
            self._chain = chain
        else:
            self._chain = []
            for i in range(num_matches):
                self._chain.append([random.choice(Environment._idx_days), random.choice(Environment._idx_hours)])


    def get_applciable_chain(self):
        l = []
        for (day, hour) in self._chain:
            l.append([Environment._days[day], Environment._hours[hour]])

        return l


    def mutate(self, partial=False):
        chrom = random.choice(range(len(self._chain)))

        if partial: #Mutate only one part of the chromosome
            idx_chrom = random.choice([0, 1]) #only this two possible indexes
            if idx_chrom == 0: #Days index
                self._chain[chrom][idx_chrom] = random.choice(Environment._idx_days) #Mutate the day
            else:
                self._chain[chrom][idx_chrom] = random.choice(Environment._idx_hours) #Mutate the hour
        else:
            self._chain[chrom] = [random.choice(Environment._idx_days), random.choice(Environment._idx_hours)] #Mutate the full chromosome


    def cross(self, genotype):
        g1 = copy.deepcopy(self._chain)
        g2 = copy.deepcopy(genotype._chain)

        sl = random.choice(range(len(g1)))
        first = random.choice([0, 1])

        g3 = []
        if first == 0:
            g3 = g1[:sl] + g2[sl:]
        else:
            g3 = g2[:sl] + g1[sl:]

        return GenotypeTimetable(chain=g3)
        

    def count(self, chrom):
        return self._chain.count(chrom)

    def __str__(self):
        return '{}'.format(self.get_applciable_chain())

    def __repr__(self):
        return '{}'.format(self.get_applciable_chain())

    def __getitem__(self, idx):
        return self._chain[idx]

    def __len__(self):
        return len(self._chain)



class FenotypeTimetable(object):
    def __init__(self, matchCombinations):
        '''
        :param matchCombinations: List of lists of combinations of matches (e.g: [[A, A], [A, B]])
        '''
        self._suitable = True
        self._m_comb = []
        for (cat1, cat2) in matchCombinations:
            self._m_comb.append([Environment._categories.index(cat1), Environment._categories.index(cat2)])


    def set_genotype(self, genotype):
        self._gen = genotype
        self._matches = []
        for i, (cat1, cat2) in enumerate(self._m_comb):
            (day, hour) = genotype[i]
            self._matches.append(Match(cat1, cat2, day, hour))


    def get_score(self):
        acum = 0
        for match in self._matches:
            raw_time = match.get_raw_time()
            coincidences = self._gen.count(raw_time) - 1
            coin_coef = Environment._concurrent_loss[coincidences]
            sc = match.get_score()
            acum += sc * coin_coef
            if sc == 0:
                self._suitable = False

        return acum

    def is_suitable(self):
        return self._suitable




gen1 = GenotypeTimetable(5)
gen2 = GenotypeTimetable(5)
print('Gen 1:', gen1)
print('Gen 2:', gen2)

print()
gen3 = gen1.cross(gen2)
print('Gen 1 + 2 (3)   :', gen3)
gen3.mutate(partial=False)
print('Gen 3 (mutation):', gen3)
print()
fen = FenotypeTimetable([['A', 'A'], ['B', 'A'], ['A', 'C'], ['A', 'B'], ['C', 'C']])
fen.set_genotype(gen3)
match1 = fen._matches[0]
print('Match 1 info: ', match1.get_home_category(), match1.get_away_category(), match1.get_base(), match1.get_hour_coef(), match1.get_score())
print('Match scores: ', [i.get_score() for i in fen._matches])
print('Total score (fitness): ', fen.get_score())

Gen 1: [['V', 18], ['L', 16], ['L', 18], ['L', 18], ['S', 16]]
Gen 2: [['S', 20], ['D', 16], ['V', 12], ['D', 12], ['S', 16]]

Gen 1 + 2 (3)   : [['V', 18], ['L', 16], ['L', 18], ['D', 12], ['S', 16]]
Gen 3 (mutation): [['V', 18], ['L', 16], ['L', 18], ['D', 18], ['S', 16]]

Match 1 info:  A A 2 0 0
Match scores:  [0, 0.0, 0, 0.65, 0.32899999999999996]
Total score (fitness):  0.979


In [39]:
class God(object):
    def __init__(self, matches):
        self.MATCHES = matches
        self.cond = Condition()
        self.STOP = False

        self.genotypes = []
        self.score_list = []
        self.suitables = []

        self.done_list = []
        self.fitness_list = []

        self.threads = []

        self.cross_genotype = None
        self.cross_fenotype = None

        self.best_genotype = None
        self.best_score = -1


    def wait_until_all_complete(self):
        while self.done_list.count(False) != 0: #wait for the threads to compute the fitness score
            pass



    def notify(self):
        self.done_list = [True] * len(self.done_list)
        with self.cond:
            self.cond.notifyAll() #notify the threads


    def living_thing(self, shared_index):
        while not self.STOP:
            fen = FenotypeTimetable(self.MATCHES)
            fen.set_genotype(self.genotypes[shared_index])
            self.score_list[shared_index] = fen.get_score()

            self.suitables[shared_index] = fen.is_suitable()
            self.done_list[shared_index] = True
            with self.cond:
                self.cond.wait()


    def start_threads(self, num_threads=8):
        l = len(self.MATCHES)
        for i in range(num_threads):
            self.genotypes.append(GenotypeTimetable(l))
            self.score_list.append(False)
            self.suitables.append(False)
            self.done_list.append(False)

            T = Thread(target=self.living_thing, args=(i,))
            self.threads.append(T)
            T.start()


    def fitness(self):
        mi = min(self.score_list)
        ma = max(self.score_list)
        l = len(self.score_list)

        self.fitness_list.clear()
        for i in range(l):
            norm = (self.score_list[i] - mi) / (ma - mi)
            norm = int(norm * 100)
            self.fitness_list.append(1 if norm == 0 else norm)


    def selection_and_cross(self):
        l = [] #weighted list for probability choice
        for i in range(len(self.genotypes)):
            l = l + [i] * self.fitness_list[i]

        gen3 = self.genotypes[random.choice(l)].cross(self.genotypes[random.choice(l)]) #probabilistic choice and cross
        self.cross_genotype = gen3
        self.cross_fenotype = FenotypeTimetable(self.MATCHES)


    def mutation(self):
        for gen in self.genotypes:
            gen.mutate() #mutation of genes

        self.cross_genotype.mutate()
        self.cross_fenotype.set_genotype(self.cross_genotype)
        
        #Add this new living thing to the population
        self.genotypes.append(self.cross_genotype)
        self.score_list.append(self.cross_fenotype.get_score())


    def reselection(self):
        l = [] #weighted list for probability choice
        for i in range(len(self.genotypes)):
            l = l + [i] * (101 - self.fitness_list[i])

        i = random.choice(l)
        
        #kill this living thing
        del self.genotypes[i]
        del self.score_list[i]


    def select_solution(self):
        for i, v in enumerate(self.suitables):
            if v != False:
                sc = self.score_list[i]
                if sc > self.best_score:
                    self.best_score = sc
                    self.best_genotype = self.genotypes[i]


    def start_life(self, living_things=8, iterations=50):
        self.STOP = False
        self.start_threads(living_things) #initiate threads

        self.wait_until_all_complete()
        self.fitness() #fitness

        #loop
        print('[', end='')
        _s = 0
        step = iterations / 50
        for i in range(iterations):
            self.selection_and_cross() #selection and cross
            self.mutation() #mutation

            self.notify()
            self.wait_until_all_complete()

            self.fitness() #fitness
            self.reselection() #reselection
            self.select_solution() #search for a solution
            if _s >= step:
                print('=',  end='')
                _s = 0

            _s += 1


        print('>] DONE!')

        self.STOP = True


god = God([['B', 'B'], ['B', 'C'], ['B', 'C'], ['B', 'C'], ['B', 'C'], ['B', 'C'], ['B', 'C'], ['B', 'A'], ['B', 'B'], ['A', 'A']])
god.start_life(iterations=10000000)
print('Best score: ', god.best_score)
print('Best genotype: ', god.best_genotype)

[=================================================>] DONE!
Best score:  6.918749999999999
Best genotype:  [['V', 16], ['S', 18], ['S', 12], ['S', 12], ['S', 20], ['D', 18], ['D', 18], ['S', 20], ['S', 18], ['V', 20]]
